In [34]:
import os
import transformers
from transformers import RobertaTokenizer, RobertaModel,AutoModelForCausalLM, AutoModelForMaskedLM,AutoModelForSequenceClassification,Trainer, TrainingArguments,AutoTokenizer
print(transformers.__version__)
from huggingface_hub import notebook_login
import wandb
from datasets import load_dataset


4.26.1


In [61]:
import torch
torch.cuda.empty_cache()

In [35]:
if not (cwd:=os.getcwd().split('\\')[-1]) == 'Twitter-Emotion-Covid-Project':
    os.chdir('\\'.join(os.getcwd().split('\\')[:-1]))
    assert os.getcwd().split('\\')[-1] == 'Twitter-Emotion-Covid-Project', 'Working directory is wrong'
print(f'Parent folder is set to: {os.getcwd()}')

Parent folder is set to: c:\Users\magnu\VSC_projects\Twitter-Emotion-Covid-Project


In [36]:
# notebook_login()

In [37]:
# wandb.login()

In [38]:
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="TwitterEmotions"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="end"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

os.environ["WANDB_NOTEBOOK_NAME"]="replicate_tut.ipynb"


In [39]:
model_checkpoint = "cardiffnlp/twitter-roberta-base-sentiment-latest"

In [40]:
datasets = load_dataset("csv", data_files='preprocessing/splits/pilot_text_only_split.csv')


Found cached dataset csv (C:/Users/magnu/.cache/huggingface/datasets/csv/default-c7030213326f81ae/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
len(datasets['train'])

3487017

In [42]:

train_size = int(len(datasets['train'])*0.10)
test_size = int(0.1 * train_size)
train_size -=test_size


In [43]:

datasets = datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
datasets


Loading cached split indices for dataset at C:\Users\magnu\.cache\huggingface\datasets\csv\default-c7030213326f81ae\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ad8bafb7a4e248b7.arrow and C:\Users\magnu\.cache\huggingface\datasets\csv\default-c7030213326f81ae\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-59448efb55e46a9a.arrow


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 313831
    })
    test: Dataset({
        features: ['text'],
        num_rows: 34870
    })
})

In [44]:
# from datasets import ClassLabel
# import random
# import pandas as pd
# from IPython.display import display, HTML

# def show_random_elements(dataset, num_examples=10):
#     assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
#     picks = []
#     for _ in range(num_examples):
#         pick = random.randint(0, len(dataset)-1)
#         while pick in picks:
#             pick = random.randint(0, len(dataset)-1)
#         picks.append(pick)
    
#     df = pd.DataFrame(dataset[picks])
#     for column, typ in dataset.features.items():
#         if isinstance(typ, ClassLabel):
#             df[column] = df[column].transform(lambda i: typ.names[i])
#     display(HTML(df.to_html()))

# show_random_elements(datasets["train"])

In [47]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", use_fast=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\magnu/.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest\snapshots\4ba3d4463bd152c9e4abd892b50844f30c646708\config.json
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "n

In [48]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [49]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

Loading cached processed dataset at C:\Users\magnu\.cache\huggingface\datasets\csv\default-c7030213326f81ae\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-003fe3cda8104aba.arrow
Loading cached processed dataset at C:\Users\magnu\.cache\huggingface\datasets\csv\default-c7030213326f81ae\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-33fbf8bdf1bcd4b1.arrow


In [50]:
block_size = 128

In [51]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [52]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
)

Loading cached processed dataset at C:\Users\magnu\.cache\huggingface\datasets\csv\default-c7030213326f81ae\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-2b9e1762a5abc9de.arrow
Loading cached processed dataset at C:\Users\magnu\.cache\huggingface\datasets\csv\default-c7030213326f81ae\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0444928cf673fa73.arrow


In [53]:
# lm_datasets.save_to_disk('tokenized_pilot_with_split.hf')

In [54]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

loading configuration file ./finetuned_roberta_pilot_v0.1/checkpoint-19500\config.json
Model config RobertaConfig {
  "_name_or_path": "./finetuned_roberta_pilot_v0.1/checkpoint-19500",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_s

In [55]:
model_name = model_checkpoint.split("/")[-1]
model_name


'checkpoint-19500'

In [56]:
training_args = TrainingArguments(
    f"finetuned_roberta_pilot_v0.1",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="wandb",
    run_name= "demo_run_1",
    # push_to_hub=True,
    num_train_epochs = 2,
    per_device_train_batch_size  = 12,
)

PyTorch: setting up devices


In [57]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [58]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
)

In [62]:
trainer.train(model_checkpoint)
wandb.finish()

Loading model from ./finetuned_roberta_pilot_v0.1/checkpoint-19500.
***** Running training *****
  Num examples = 119070
  Num Epochs = 2
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 19846
  Number of trainable parameters = 124697433
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 19500
  Will skip the first 1 epochs then the first 9577 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/9577 [00:00<?, ?it/s]

  0%|          | 0/19846 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 13259
  Batch size = 8


  0%|          | 0/1658 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to C:\Users\magnu\AppData\Local\Temp\tmp_lw_8bci
Configuration saved in C:\Users\magnu\AppData\Local\Temp\tmp_lw_8bci\config.json


{'eval_loss': 2.56998872756958, 'eval_runtime': 132.5836, 'eval_samples_per_second': 100.005, 'eval_steps_per_second': 12.505, 'epoch': 2.0}
{'train_runtime': 316.0707, 'train_samples_per_second': 753.439, 'train_steps_per_second': 62.79, 'train_loss': 0.04851299094827043, 'epoch': 2.0}


Model weights saved in C:\Users\magnu\AppData\Local\Temp\tmp_lw_8bci\pytorch_model.bin
Logging model artifacts. ...


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁██
train/global_step,▁██
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [63]:
trainer.push_to_hub()

c:\Users\magnu\VSC_projects\Twitter-Emotion-Covid-Project\finetuned_roberta_pilot_v0.1 is already a clone of https://huggingface.co/Magdk01/finetuned_roberta_pilot_v0.1. Make sure you pull the latest changes with `repo.git_pull()`.


OSError: From https://huggingface.co/Magdk01/finetuned_roberta_pilot_v0.1
   f4df5d6..5992077  main       -> origin/main
fatal: refusing to merge unrelated histories
